In [53]:
import pandas as pd
import os
import unicodedata
from config import DATA_DIR

In [54]:
csv_path = os.path.join(DATA_DIR, 'clean_scraped_services.csv')
df = pd.read_csv(csv_path)

In [55]:
df.head()

,permite_pedido_anomimo,img_url,service_desc,service_id,ativo,subtheme_name,theme_name,description
0,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Adotar cães e gatos,3676,True,Adoção de animais,Animais,O QUE É\nÉ a adoção de cães e gatos alojados n...
1,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Pombos - Solicitar vistoria em local infestado,816,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...
2,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Morcegos - Reclamar sobre local com morcegos,815,True,Animais que transmitem doenças ou risco à saúde,Animais,"O QUE É\nOrientação, esclarecimento de dúvidas..."
3,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Ratos - Solicitar vistoria em local infestado,814,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nSolicitação de vistoria em local com ...
4,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Escorpião - Solicitar vistoria de local com es...,813,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...


In [56]:
exemplo = df.loc[0, 'description']

In [57]:
def remover_acentos(texto: str) -> str:

    nfkd = unicodedata.normalize("NFD", texto)
    texto_sem_acentos = "".join(
        c for c in nfkd if unicodedata.category(c) != "Mn"
    )
    
    return texto_sem_acentos

def clean_chave(chave:str)->str:

    chave = chave.lower().replace(' ', '_').replace('-', '_').replace(',', '_').replace('__', '_').strip()
    chave = remover_acentos(chave)

    return chave

def clean_pedaco(pedaco:str)->str:

    pedaco = pedaco.strip().replace('\xa0', '').replace('clique aqui', '')
    return pedaco


def parse_description(description:str)->dict[str, str]:

    pedacos = description.split('\n')

    atualizacao = pedacos.pop(-1).replace('Atualizado em:', '').strip()
    criacao  = pedacos.pop(-1).replace('Criado em:', '').strip()

    parsed = {
        'dt_atualizacao' : atualizacao,
        'dt_criacao'    : criacao,
    }

    chave = None
    for i, pedaco in enumerate(pedacos):
        if pedaco.isupper():
            chave = clean_chave(pedaco)
        else:
            if chave:
                if chave not in parsed:
                    parsed[chave] = clean_pedaco(pedaco)
                else:
                    parsed[chave] =  parsed[chave] + '***|||***' + clean_pedaco(pedaco)
    
    return parsed


def chunk_header(chunk:str, chunk_key: str, service_name:str, posit:int|None=None)->str:

    header = f"[{chunk_key.upper()} | {service_name}]"

    if posit is not None:
        header = f"[{chunk_key.upper()} | {posit} | {service_name}]"

    return header + '\n' + chunk


def solve_list_chunks(parsed:dict, chave:str, service_name:str, splited:list[str])->None:

    parsed[chave] = []
    pular_next= False
    posit = 1
    for i, item in enumerate(splited):
        if pular_next:
            pular_next= False
            continue
        if item.endswith(':') and i+1 < len(splited):
            pular_next = True
            item = item + ' ' + splited[i+1]
        parsed[chave].append(chunk_header(item, chave, service_name, posit))
        posit+=1

def add_header_to_chunks(parsed_chunks:dict[str, str], service_name:str)->dict[str, str]:


    parsed = {}
    #reestruturando os chunks para formato de lista e colocando o header
    for chave, pedacos in parsed_chunks.items():

        splited = pedacos.split('***|||***')
        if len(splited)>1:
            solve_list_chunks(parsed, chave, service_name, splited)
        else:
            parsed[chave] = chunk_header(pedacos, chave, service_name)

    return parsed


def chunk_json_pipeline(description:str, service_name:str)->dict[str, str]:

    parsed = parse_description(description)
    parsed_with_headers = add_header_to_chunks(parsed, service_name)

    return parsed_with_headers


In [58]:
parsed_exemplo = chunk_json_pipeline(exemplo, 'Exemplo de Serviço')
parsed_exemplo

{'dt_atualizacao': '[DT_ATUALIZACAO | Exemplo de Serviço]\n20/10/2025',
 'dt_criacao': '[DT_CRIACAO | Exemplo de Serviço]\n23/04/2020',
 'o_que_e': '[O_QUE_E | Exemplo de Serviço]\nÉ a adoção de cães e gatos alojados no Centro Municipal de Adoção. Os animais disponíveis estão castrados, vacinados, vermifugados e microchipados.',
 'quando_solicitar': '[QUANDO_SOLICITAR | Exemplo de Serviço]\nQuando uma pessoa deseja realizar a adoção de um cão ou gato, consciente de que será o responsável pelo animal em todas as etapas de sua vida, promovendo a guarda responsável, o que inclui alimentação adequada, água, higiene, vacinação, cuidados médico-veterinários, atenção e muito carinho.',
 'publico_alvo': '[PUBLICO_ALVO | Exemplo de Serviço]\nPessoas maiores de idade que desejam adotar cães ou gatos.',
 'requisitos_documentos_e_informacoes': ['[REQUISITOS_DOCUMENTOS_E_INFORMACOES | 1 | Exemplo de Serviço]\nRequisitos necessários para a solicitação: - Ser maior de idade;',
  '[REQUISITOS_DOCUMENT

In [59]:
def flaten_chunks(parsed_json:dict)->list[str]:

    flatened = []
    for pedacos in parsed_json.values():
        if isinstance(pedacos, list):
            for pedaco in pedacos:
                flatened.append(pedaco)
        else:
            flatened.append(pedacos)
    
    return flatened

In [60]:
chunks = flaten_chunks(parsed_exemplo)

In [61]:
chunks[0]

'[DT_ATUALIZACAO | Exemplo de Serviço]\n20/10/2025'

In [62]:
df.head()

,permite_pedido_anomimo,img_url,service_desc,service_id,ativo,subtheme_name,theme_name,description
0,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Adotar cães e gatos,3676,True,Adoção de animais,Animais,O QUE É\nÉ a adoção de cães e gatos alojados n...
1,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Pombos - Solicitar vistoria em local infestado,816,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...
2,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Morcegos - Reclamar sobre local com morcegos,815,True,Animais que transmitem doenças ou risco à saúde,Animais,"O QUE É\nOrientação, esclarecimento de dúvidas..."
3,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Ratos - Solicitar vistoria em local infestado,814,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nSolicitação de vistoria em local com ...
4,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Escorpião - Solicitar vistoria de local com es...,813,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...


In [63]:
all_chunks = []
metadata_list = []
for i, row in df.iterrows():
    description = row['description']
    service_name = row['service_desc']
    parsed_chunks = chunk_json_pipeline(description, service_name)
    chunks = flaten_chunks(parsed_chunks)
    all_chunks.extend(chunks)
    metadata_list.extend([{
                           "service_name": service_name,
                           "service_id" : row['service_id'],
                            "theme" : row['theme_name'],
                            "subtheme" : row['subtheme_name']
                                              }] * len(chunks))

In [64]:
assert len(all_chunks) == len(metadata_list)

In [65]:
import json

In [66]:
dados = {
    "chunks": all_chunks,
    "metadata": metadata_list}

In [67]:
json_path = os.path.join(DATA_DIR, 'chunks_metadata.json')
with open(json_path, 'w') as f:
    json.dump(dados, f, ensure_ascii=False, indent=4)